# Template Generation for Testing CalB/C Models

This code is designed to generate templates for testing CalB/C models. The template generation consists of duplicating specific sheets from a source model, renaming them with version numbers (1.0, 2.0....) and updating their named ranges. For further details, refer to `template_generation.md`. 



In [ ]:
import openpyxl
import xlwings as xw
import pandas as pd
import shutil
import io
import os
import pandas as pd
import time

In [ ]:
user_profile = os.getenv('USERPROFILE') ## Retrieving user profile directory from 'USERPROFILE' environment variable.

In [ ]:
# Constructing the base directory path using the user profile directory.
base_dir = os.path.join(
    user_profile, 
    'California Department of Transportation',
    'DOT HQ PMP Cal B C Update - General',
    'Testbed',
    'Input'
)

In [ ]:
at_model = os.path.join(base_dir, 'CalBC Models', 'AT_model.xlsm')
sketch_model = os.path.join(base_dir, 'CalBC Models', 'Sketch_model.xlsm')
if_model = os.path.join(base_dir, 'CalBC Models', 'Intermodal_freight_model.xlsm')
pr_model = os.path.join(base_dir, 'CalBC Models', 'Park_&_ride_model.xlsm')
corridor_model = os.path.join(base_dir, 'CalBC Models', 'Corridor_model.xlsm')



In [ ]:
main_model = at_model # Specify the model here 

sheets_to_copy = ['1a) Project Info', '1b) Non-Inf Program Info', '2) Model Inputs']  # List of sheets to copy
num_replicas = 2  # Specify the number of replications needed




In [ ]:
def copy_sheets_with_unique_named_ranges(main_model, sheets_to_copy, num_replicas, base_dir):
    """
    Function to copy sheets and update named ranges with unique names.
    """
    # Open the workbook
    with xw.App(visible=True) as app:
        try:
            wb = app.books.open(main_model)

            # Iterate over each sheet specified above
            for sheet_name in sheets_to_copy:
                original_sheet = wb.sheets[sheet_name]
                
                # Extract the base sheet name without the version suffix (e.g., "Project_info" from "Project_info_1.0")
                base_name = sheet_name.rsplit('.', 1)[0]

                for i in range(2, num_replicas + 1):
                    # Create the copied sheet
                    copied_sheet = original_sheet.copy(after=original_sheet)

                    # Rename the copied sheet (e.g., Project_info_1.0, Project_info_2.0, ...)
                    copied_sheet.name = f"{base_name}_{i}.0"
                    
                    # Get all named ranges in the workbook
                    named_ranges = wb.names

                    # Iterate through the named ranges
                    for named_range in named_ranges:
                        try:
                            if named_range.refers_to_range:
                                if named_range.refers_to_range.sheet == copied_sheet:
                                    original_name = named_range.name
                                    original_range = named_range.refers_to_range

                                    # Create the new name by appending the suffix (2.0, 3.0, etc.)
                                    new_name = f"{original_name}{i}.0" if not original_name.endswith(f"{i}.0") else original_name

                                    # Delete the existing named range in the copied sheet
                                    named_range.delete()

                                    # Create a new named range in the copied sheet
                                    copied_sheet.range(original_range.address).name = new_name
                        except Exception as e:
                            print(f"Skipping named range {named_range.name} due to error: {e}")
                            continue


            # Define the path for the new workbook
            output_filename = f"{os.path.splitext(os.path.basename(main_model))[0]}_template.xlsx"
            output_file = os.path.join(base_dir, 'Templates', output_filename)

            # Save the workbook with the changes
            wb.save(output_file)

        except Exception as e:
            print(f"Error while processing: {e}")
        finally:
            # Make sure the workbook is closed and the Excel application quits
            wb.close()





In [ ]:
copy_sheets_with_unique_named_ranges(main_model, sheets_to_copy, num_replicas, base_dir)